In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/food-cmpe260/RAW_interactions.csv


In [2]:
!pip install faiss-gpu
!pip install -U sentence-transformers

     |████████████████████████████████| 85.5 MB 130 kB/s  eta 0:00:01    |█████████                       | 24.0 MB 4.4 MB/s eta 0:00:14     |████████████▍                   | 33.0 MB 4.4 MB/s eta 0:00:12     |████████████████████████        | 64.1 MB 52.8 MB/s eta 0:00:01     |█████████████████████████▍      | 67.8 MB 52.8 MB/s eta 0:00:01     |██████████████████████████▊     | 71.4 MB 52.8 MB/s eta 0:00:01     |███████████████████████████▍    | 73.3 MB 52.8 MB/s eta 0:00:01     |████████████████████████████▏   | 75.2 MB 52.8 MB/s eta 0:00:01
     |████████████████████████████████| 85 kB 2.4 MB/s eta 0:00:011
     |████████████████████████████████| 224 kB 17.2 MB/s eta 0:00:01
     |████████████████████████████████| 7.0 MB 57.7 MB/s eta 0:00:01
     |████████████████████████████████| 7.8 MB 53.1 MB/s eta 0:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125918 sha256=f60dc577c32ad8dc5d1623dbdf8730a473ac4b9307c29d4b54bafdb2b37

In [3]:
import numpy as np 
import pandas as pd

In [4]:
import math

In [5]:
import faiss
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [6]:
import pandas as pd

food = pd.read_csv('../input/food-cmpe260/RAW_interactions.csv')

In [7]:
food=food[pd.notnull(food['review'])]
food=food.reset_index(drop=True)

food['id']=food.index

In [8]:
sentences = food['review'].tolist()
print(len(sentences))

1132198


In [9]:
embeddings = model.encode(sentences)
faiss.normalize_L2(embeddings)

Batches:   0%|          | 0/35382 [00:00<?, ?it/s]

In [10]:
print(embeddings.shape)

(1132198, 768)


In [11]:
dim=768
ncentroids=50 
m=16 
quantiser = faiss.IndexFlatL2(dim)
index = faiss.IndexIVFPQ (quantiser, dim,ncentroids, m , 8)
index.train(embeddings) 
print(index.is_trained)
faiss.write_index(index, "trained.index")

True


In [12]:
ids = food['id'].tolist()
ids = np.array(ids)
index.add_with_ids(embeddings,ids)
print(index.ntotal)

1132198


In [13]:
faiss.write_index(index,"block.index")

In [14]:
def searchFAISSIndex(data,id_col_name,query,index,nprobe,model,topk=20):
    
    query_embedding=model.encode([query])[0]

    dim=query_embedding.shape[0]
    query_embedding=query_embedding.reshape(1,dim)
    faiss.normalize_L2(query_embedding)
    index.nprobe=nprobe

    D,I=index.search(query_embedding,topk) 
    ids=[i for i in I][0]
    L2_score=[d for d in D][0]
    inner_product=[calculateInnerProduct(l2) for l2 in L2_score]
    
    search_result=pd.DataFrame()
    search_result[id_col_name]=ids
    search_result['cosine_sim']=inner_product
    search_result['L2_score']=L2_score
    dat=data[data[id_col_name].isin(ids)]
    dat=pd.merge(dat,search_result,on=id_col_name)
    dat=dat.sort_values('cosine_sim',ascending=False)
    return dat

In [15]:
def calculateInnerProduct(L2_score):
    return (2-math.pow(L2_score,2))/2

In [19]:
query = input("Enter Query: ")
search_result=searchFAISSIndex(food,"id",query,index,nprobe=10,model=model,topk=20)
search_result=search_result[['id','review','rating','cosine_sim','L2_score']]
search_result

Enter Query:  cheeese


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,id,review,rating,cosine_sim,L2_score
12,476189,The term pasteles have a few different meanin...,0,0.614119,0.878500
16,994162,A wonderful Brownie made from scratch.,0,0.573308,0.923787
1,39123,choo-moose? its hummus,0,0.569206,0.928217
8,273945,This is one of THE best cheesecakes I have eve...,5,0.567686,0.929854
0,316,White land o lakes is a white american cheese,0,0.565104,0.932626
7,271553,mmmmmmmmmmm cheesey,4,0.564584,0.933184
19,1115949,The recepie calls for cheesebut do not say whe...,1,0.559596,0.938513
13,630255,but i think there is melted cheeses in the pho...,0,0.557829,0.940395
17,995395,"Gesundheit, I mean danke for the schnitzel rec...",5,0.555832,0.942516
9,355420,meatloaf on a bun. interesting.,0,0.553649,0.944829


In [ ]:
search_result